Below is the importing of all relevant libraries for this specific document.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sma

Below reads in all the cleaned data created in other notebooks and then subsets it to only include drugs with a price greater than 0.

In [2]:
df = pd.read_csv("CleanedData.csv")
df = df[df["Price"] > 0]

Below is a quick info to just check the datatypes of the cleaned columns, and again validate that there are no null values in the data.

In [3]:
df.info()

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

Below is the generation of the summary statistics of Inflation Adjusted Price, which is then used to create the Z score of inflation adjusted price for implementation in our models. We also filter out all prices which exceed 3 standard deviations in either direction from the mean.

In [4]:
# These summary stats are generated for use in the formula present in the next block.
mean = df['InflationAdjustedPrice'].mean()
print(f"mean for the inflation adjusted price: {mean}")
std =df['InflationAdjustedPrice'].std()
print(f"std for the inflation adjusted price: {std} \n")

# Here we generate the z-score for the variable, and compare the size from before and after the swap.
df['InflationAdjustedPriceZScore'] = (df['InflationAdjustedPrice'] - mean)/std
print(f"Size before z-score filter:{df.shape[0]}")
df= df[(df['InflationAdjustedPriceZScore'] <=3)&(df['InflationAdjustedPriceZScore'] >=-3)]
print(f"Size after z-score filter:{df.shape[0]}")

mean for the inflation adjusted price: 925.5657900172896
std for the inflation adjusted price: 2226.7695815899065 

Size before z-score filter:1643
Size after z-score filter:1584


Here we're just looking at the distribution between our target variable classes after this filter has been applied.

In [5]:
df['P or E'].value_counts(normalize=True)

P    0.806187
E    0.193813
Name: P or E, dtype: float64

Assigning binary dependent variable from the Analysis variable. The following categories will be classified as suspected evergreen:
- P:PED
- PTAorPTE
- DlistRequest
- NPP
- PED
- P-PEDExtension
- UCsamemonth
- DP
- DS/DP
- DS/DP/UCnew
- DP/UCnew
- DS

Create a new column with a direct evergreen flag, whose value is determined solely by the 'E' flag being present in the 'P or E' column,

In [6]:
df['EvergreenFlag'] = [0] * len(df)
df.loc[df['P or E']=='E','EvergreenFlag'] = 1

C:\Users\iceti\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\iceti\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


We recount the distribution, just to make sure it's the same.

In [7]:
df['EvergreenFlag'].value_counts(normalize=True)

0    0.806187
1    0.193813
Name: EvergreenFlag, dtype: float64

This is the creation of a specific dataframe for use in the model, which includes the following variables:

* Inflation Adjusted Price (InflationAdjustedPrice)
* Months Until Exclusivity Expiration (MonthsUntilExpiration)
* Number of Previous Patent Filings (PreviousPatents)
* Evergreen Flag (EvergreenFlag)

Please add any variable changes to this list to reflect the current conceptual model.

In [8]:
model_df = df[['InflationAdjustedPrice','MonthsUntilExpiration',
              'PreviousPatents','EvergreenFlag']]
#model_df['Initial Filing'] = [0] * len(model_df)
#model_df.loc[model_df['PreviousPatents']==0,'Initial Filing'] = 1

Just another quick info run to check on our variables and distribution.

In [9]:
model_df.info()

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

This is the creation of a quick function which standardizes a numberical column to a z-score representation of it's values. This is used subsequently.

In [10]:
def ZStandardize(array):
    mean = array.mean()
    std = array.std()
    new_array = (array - mean)/std
    return(new_array)

Here we standardize all the columns except for the binary target using the above z-score conversion.

In [11]:
for column in model_df.drop(columns='EvergreenFlag').columns:
    model_df[column] = ZStandardize(model_df[column])

C:\Users\iceti\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Here we just re-run and check the function worked correctly.

In [12]:
model_df.describe()

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

Here we check the target variable proportions again.

In [13]:
model_df['EvergreenFlag'].value_counts(normalize=True)

0    0.806187
1    0.193813
Name: EvergreenFlag, dtype: float64

Here we run a quick stats-models-api logistic regression model for initial analysis.

In [14]:
model_df = sma.add_constant(model_df)
clf = sma.Logit(model_df['EvergreenFlag'],model_df.drop(columns='EvergreenFlag')).fit()
print(clf.summary())

Optimization terminated successfully.
         Current function value: 0.475172
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:          EvergreenFlag   No. Observations:                 1584
Model:                          Logit   Df Residuals:                     1580
Method:                           MLE   Df Model:                            3
Date:                Tue, 28 Mar 2023   Pseudo R-squ.:                 0.03362
Time:                        11:23:59   Log-Likelihood:                -752.67
converged:                       True   LL-Null:                       -778.86
Covariance Type:            nonrobust   LLR p-value:                 2.490e-11
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -1.4856      0.067    -22.251      0.000      -1.616      

C:\Users\iceti\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2608: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [15]:
df.to_csv('ModelScripts/ModelData/mbdata.csv')